In [38]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data_path = "E:\\DataSets\\keisci\\sensor\\sensor_train\\"

In [29]:
data = pd.read_csv(data_path+"sensor_train.csv")
data.head(2)

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0


# 根据数据总结出来的结论
+ fragment_id 总共有7300条 
+ 每个fragment_id 对应一个behavior_id
+ 每个fragment_id对应的数据条数： max:61, mean:58, min:50
+ time_point 是每个fragment_id采集的时间点, 每个fragment_id是独立的,范围[0,49999]
+ 其他各列的取值范围 acc_x:(-14.7, 34.5) &ensp;   acc_y:(-14.7, 13.4) &ensp;   acc_z:(-12.6, 28.2) &ensp;   acc_xg:(-19.8, 30.2) &ensp;   acc_yg:(-16.9, 14.4) &ensp;   acc_zg:(-8.9, 38.1)

**goupby("behavior_id"), countdistinct("fragment_id")**

In [39]:
behavior_id_ct = data.pivot_table(index='behavior_id',values='fragment_id',aggfunc=lambda x:len(x.unique())).to_dict()

In [51]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Bar


bar = Bar()\
.add_xaxis(list(behavior_id_ct["fragment_id"].keys()))\
.add_yaxis("fragment_id 数量", list(behavior_id_ct["fragment_id"].values()))\
.set_global_opts(title_opts=opts.TitleOpts(title="behavioro_id 对应的fragment_id数量 ", subtitle=""))\
.set_series_opts(
    label_opts=opts.LabelOpts(is_show=True),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
)

# 需要在第一次渲染时加载javascript文件 
# bar.load_javascript()
bar.render_notebook()

C:\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


**绘制每个列根据时间的变化情况**

In [73]:
data_0 = data[data["fragment_id"]==0].to_dict()